In [1]:
import csv 
import pandas as pd
# 按照每门课程的考试班级数目进行排序
def SamplingCourse(file):
    with open(file,"r",encoding="UTF-8") as f:
        reader = csv.reader(f)
        column = [row[0] for row in reader]
    hashmapClass = {}
    for i in range(len(column)):
        hashmapClass[column[i]] = hashmapClass.get(column[i],0) + 1
    sortedCourse = sorted(hashmapClass.items(), key=lambda d:d[1], reverse = True)
    return sortedCourse[15:25]

In [2]:
# 生成课程和班级的表
data = pd.read_csv("classs_inf.csv", usecols=["ci_course_no","ci_class_name","ci_student_number"])

In [3]:
def findCommonTime(temp, listClass):
    commonTime = []
    for i in range(len(temp)):
        commonTime.append(temp[i] & listClass[i])
    return commonTime

In [4]:
def writeToCsv(temp, course, classes,number):
    course = [course] * len(classes)
    time = [temp.index(1)] * len(classes)
    test = pd.DataFrame({'课程': course, "考试时间": time, "班级": classes,"班级人数": number})
    test.to_csv("temp.csv",encoding = "utf-8",mode="a",index = None, header = False)

In [5]:
def updateClassTime(dict, listclass, index):
    for classes in listclass:
        dict[classes][index] = 0
    return dict


In [9]:
# 假设总共考3天，每天4个考试时间段
# 生成一个字典，对应的形式是这样的{"班级1":[],"班级2":[],"班级3":[],...}
dictClassTime = {}
for courseNumber in SamplingCourse("classs_inf.csv"):
    course = courseNumber[0]
    # 查找每一个考这门课的班级
    temp = [1]*12
    classWhoTakeCourse = []
    classCourseNumber = []
    for i in range(len(data)):
        if data.loc[i]["ci_course_no"] == course:
            className = data.loc[i]["ci_class_name"]
            classNumber = data.loc[i]["ci_student_number"]
            # 对考这门课的班级的时间进行初始化, 如果不存在就生成一个[1,1,1,1...,]
            dictClassTime[className] = dictClassTime.get(className,[1]*12)
            # 需要记录所有有这门课的班级
            classWhoTakeCourse.append(className)
            classCourseNumber.append(classNumber)
            # 找到所有参加这门考试 班级的共同空闲时间
            # [0,1,1,...,] 和 [1,0,1,...,] 相与
            temp = findCommonTime(temp,dictClassTime[className])
    if 1 not in temp:
        print("当前" + course +"无法排考")
    # 找到共同可行的时间, 写入csv, 并且标记占用时间
    else:
        writeToCsv(temp,course,classWhoTakeCourse,classCourseNumber)
        updateClassTime(dictClassTime, classWhoTakeCourse, temp.index(1))